# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Weather_cities = pd.read_csv("Resources/Weather_cities.csv")
Weather_cities= Weather_cities.drop(columns="Unnamed: 0")
Weather_cities

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.60,168.33,49.19,74,88,26.69,NZ,1591484330
1,Saskylakh,71.92,114.08,42.66,61,100,8.61,RU,1591484147
2,Rikitea,-23.12,-134.97,73.74,90,100,27.56,PF,1591484331
3,Wadena,46.44,-95.14,66.20,52,90,16.11,US,1591484331
4,Narsaq,60.92,-46.05,53.60,50,100,20.80,GL,1591484106
...,...,...,...,...,...,...,...,...,...
584,Odesskoye,54.21,72.97,56.28,80,7,5.23,RU,1591484610
585,Ostrov,57.34,28.35,50.00,93,14,2.24,RU,1591484375
586,Brae,60.40,-1.35,46.40,93,100,27.51,GB,1591484612
587,Dalmatovo,56.26,62.94,52.70,66,0,5.30,RU,1591484613


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# Store latitude and longitude in locations (make a subset)
locations = Weather_cities[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = Weather_cities["Humidity"].astype(float)
fig = gmaps.figure(map_type="TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=40,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
Ideal_tem = Weather_cities.loc[(Weather_cities["Max Temp"]>70) & (Weather_cities["Max Temp"]<85)]
Ideal_hum = Ideal_tem.loc[Ideal_tem["Humidity"]<70]
hotel_df = Ideal_hum.loc[Ideal_tem["Cloudiness"]==0]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df["Hotel name"]=""
hotel_df

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
0,Albany,42.60,-73.97,79.00,24,0,3.00,US,1591484103,
1,Lodi,38.13,-121.27,75.00,34,0,5.99,US,1591484108,
2,Sur,22.57,59.53,83.57,64,0,7.78,OM,1591484412,
3,Guerrero Negro,27.98,-114.06,71.53,60,0,23.62,MX,1591484164,
4,Port Keats,-14.25,129.55,71.15,58,0,10.33,AU,1591484520,
5,Sakakah,29.97,40.21,82.40,11,0,3.36,SA,1591484536,
6,Faanui,-16.48,-151.75,76.44,58,0,5.26,PF,1591484549,
7,Abu Samrah,35.30,37.18,79.21,24,0,4.38,SY,1591484584,
8,Turaif,31.67,38.66,84.20,25,0,25.28,SA,1591484585,
9,Erzin,36.96,36.20,75.99,44,0,7.00,TR,1591484603,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotels_name=[]
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Latitude"]
    lng = hotel_df.loc[i]["Longitude"]
    
    params = {"location":f"{lat},{lng}",
    "radius": 5000,
    "types": "hotel",
    "key": gkey}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotels_name.append(name_address["results"][0]["name"])
        #hotel_df.loc[i]["Hotel Name"] = name_address["results"][0]["name"]
    except:
        #hotels_name.append(name_address[""])
        print("not found")


hotel_df["Hotel Name"]=hotels_name
hotel_df

,Cities,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel Name
0,Albany,42.60,-73.97,79.00,24,0,3.00,US,1591484103,Voorheesville,Voorheesville
1,Lodi,38.13,-121.27,75.00,34,0,5.99,US,1591484108,Lodi,Lodi
2,Sur,22.57,59.53,83.57,64,0,7.78,OM,1591484412,Sur,Sur
3,Guerrero Negro,27.98,-114.06,71.53,60,0,23.62,MX,1591484164,Guerrero Negro,Guerrero Negro
4,Port Keats,-14.25,129.55,71.15,58,0,10.33,AU,1591484520,Thamarrurr,Thamarrurr
5,Sakakah,29.97,40.21,82.40,11,0,3.36,SA,1591484536,Sakaka,Sakaka
6,Faanui,-16.48,-151.75,76.44,58,0,5.26,PF,1591484549,Vaitape,Vaitape
7,Abu Samrah,35.30,37.18,79.21,24,0,4.38,SY,1591484584,Suruj,Suruj
8,Turaif,31.67,38.66,84.20,25,0,25.28,SA,1591484585,Turaif,Turaif
9,Erzin,36.96,36.20,75.99,44,0,7.00,TR,1591484603,Erzin,Erzin


In [25]:
name_address

{'html_attributions': [],
 'next_page_token': 'CqQCGgEAAKCrC_IhO9yljNOe7J-dNxSeBdPRS-Sspa7PwREx4YGPe0Dm0Gg7taGr27KbGv75DXaWX38gqFsXZpn7XNMI-tbZP2gt4h9LmF_F76Nxl35dKfTyOGz3E_fK_F-ihQ1wQub08d877aBQR9b3qsIMhaTIJZ38xSqln7Bc5olHpx4NNOVxQxwpXD2KrQq8-PzSEXzMzTqyu-dBtOCA7Z2uib19DqjZU23IRUSlnNBfNIICBHNJHPhqs3a_aMx56Z8t1DhS-mNvggH_dRXv69kKhzVL5WP4kdGDOly35SHPzy3RlsoFzQIP0ekJ8Ve8At0glPiYuvRu69qT27bMq8ahtdZoAbi0-CKfkBGJa1vpmC3QiBERUwg2quO-wCPbHt-zVBIQAwvVwca8oTx6YATcA1GiGxoUZlF_cEcfqb0TEXrocws8TvWByFg',
 'results': [{'geometry': {'location': {'lat': 36.95383899999999,
     'lng': 36.205229},
    'viewport': {'northeast': {'lat': 36.96528199999999, 'lng': 36.223954},
     'southwest': {'lat': 36.943781, 'lng': 36.174113}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
   'id': 'dd2905bf17c4190c85c89d357f2cf590672edfad',
   'name': 'Erzin',
   'photos': [{'height': 2308,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105890473574944199075"

In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [49]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
symbols = gmaps.symbol_layer(
        locations=locations2, fill_color='red', stroke_color='red', info_box_content=hotel_info)
fig.add_layer(symbols)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))